# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
housing = pd.read_csv("cal_housing_clean.csv")

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
X = housing.drop("medianHouseValue",axis = 1)

In [6]:
y = housing['medianHouseValue']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [12]:
scaler.fit(X)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [14]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [16]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [17]:
import tensorflow as tf

In [20]:
age  = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medIncome = tf.feature_column.numeric_column('medianIncome')

In [21]:
feat_cols = [ age,rooms,bedrooms,population,households,medIncome]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [24]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [25]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_986yqo9', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [26]:
model.train(input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp_986yqo9/model.ckpt.
INFO:tensorflow:loss = 632194470000.0, step = 1
INFO:tensorflow:global_step/sec: 694.372
INFO:tensorflow:loss = 253876220000.0, step = 101 (0.149 sec)
INFO:tensorflow:global_step/sec: 719.21
INFO:tensorflow:loss = 514768600000.0, step = 201 (0.137 sec)
INFO:tensorflow:global_step/sec: 520.498
INFO:tensorflow:loss = 657855300000.0, step = 301 (0.191 sec)
INFO:tensorflow:global_step/sec: 455.561
INFO:tensorflow:loss = 669232300000.0, step = 401 (0.223 sec)
INFO:tensorflow:global_step/sec: 410.905
INFO:tensorflow:loss = 584697800000.0, step = 501 (0.241 sec)
INFO:tensorflow:global_step/sec: 582.931
INFO:tensorflow:loss = 767403900000.0, step = 601 (0.170 sec)
INFO:tensorflow:global_step/sec: 615.99
INFO:tensorflow:loss = 427011150000.0, step = 701 (0.162 sec)
INFO:tensorflow:global_step/sec: 572.936
INFO:tensorflow:loss = 660988760000.0, step = 801 (0.176 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 544.68
INFO:tensorflow:loss = 65339175000.0, step = 7901 (0.182 sec)
INFO:tensorflow:global_step/sec: 520.039
INFO:tensorflow:loss = 95522900000.0, step = 8001 (0.192 sec)
INFO:tensorflow:global_step/sec: 332.252
INFO:tensorflow:loss = 40867734000.0, step = 8101 (0.302 sec)
INFO:tensorflow:global_step/sec: 557.674
INFO:tensorflow:loss = 46150353000.0, step = 8201 (0.178 sec)
INFO:tensorflow:global_step/sec: 598.135
INFO:tensorflow:loss = 72393820000.0, step = 8301 (0.168 sec)
INFO:tensorflow:global_step/sec: 616.79
INFO:tensorflow:loss = 91903000000.0, step = 8401 (0.160 sec)
INFO:tensorflow:global_step/sec: 615.529
INFO:tensorflow:loss = 130946670000.0, step = 8501 (0.164 sec)
INFO:tensorflow:global_step/sec: 572.432
INFO:tensorflow:loss = 113182170000.0, step = 8601 (0.173 sec)
INFO:tensorflow:global_step/sec: 563.3
INFO:tensorflow:loss = 137093380000.0, step = 8701 (0.177 sec)
INFO:tensorflow:global_step/sec: 554.38
INFO:tensorflow:loss = 11069120000

INFO:tensorflow:global_step/sec: 547.223
INFO:tensorflow:loss = 157041700000.0, step = 15801 (0.181 sec)
INFO:tensorflow:global_step/sec: 573.957
INFO:tensorflow:loss = 78042440000.0, step = 15901 (0.173 sec)
INFO:tensorflow:global_step/sec: 600.737
INFO:tensorflow:loss = 103360220000.0, step = 16001 (0.167 sec)
INFO:tensorflow:global_step/sec: 598.879
INFO:tensorflow:loss = 137963370000.0, step = 16101 (0.169 sec)
INFO:tensorflow:global_step/sec: 624.485
INFO:tensorflow:loss = 72667630000.0, step = 16201 (0.159 sec)
INFO:tensorflow:global_step/sec: 609.674
INFO:tensorflow:loss = 70066670000.0, step = 16301 (0.163 sec)
INFO:tensorflow:global_step/sec: 614.627
INFO:tensorflow:loss = 57641816000.0, step = 16401 (0.163 sec)
INFO:tensorflow:global_step/sec: 610.009
INFO:tensorflow:loss = 89095360000.0, step = 16501 (0.162 sec)
INFO:tensorflow:global_step/sec: 606.803
INFO:tensorflow:loss = 200261040000.0, step = 16601 (0.166 sec)
INFO:tensorflow:global_step/sec: 636.834
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 567.558
INFO:tensorflow:loss = 41109357000.0, step = 23701 (0.177 sec)
INFO:tensorflow:global_step/sec: 567.975
INFO:tensorflow:loss = 151145610000.0, step = 23801 (0.174 sec)
INFO:tensorflow:global_step/sec: 637.17
INFO:tensorflow:loss = 89978860000.0, step = 23901 (0.155 sec)
INFO:tensorflow:global_step/sec: 615.027
INFO:tensorflow:loss = 262976670000.0, step = 24001 (0.162 sec)
INFO:tensorflow:global_step/sec: 530.992
INFO:tensorflow:loss = 74045930000.0, step = 24101 (0.190 sec)
INFO:tensorflow:global_step/sec: 577.214
INFO:tensorflow:loss = 128185330000.0, step = 24201 (0.172 sec)
INFO:tensorflow:global_step/sec: 597.862
INFO:tensorflow:loss = 98182590000.0, step = 24301 (0.169 sec)
INFO:tensorflow:global_step/sec: 640.233
INFO:tensorflow:loss = 38504296000.0, step = 24401 (0.155 sec)
INFO:tensorflow:global_step/sec: 619.022
INFO:tensorflow:loss = 133934530000.0, step = 24501 (0.161 sec)
INFO:tensorflow:global_step/sec: 627.719
INFO:tensorflow:loss

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [27]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [28]:
pred_gen = model.predict(predict_input_func)

In [29]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from /tmp/tmp_986yqo9/model.ckpt-25000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [30]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
mean_squared_error(y_test,final_preds)**0.5

97574.11289878124

In [33]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size = 10,
      num_epochs=1,
      shuffle=False)

In [34]:
model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2018-11-06-07:55:09
INFO:tensorflow:Restoring parameters from /tmp/tmp_986yqo9/model.ckpt-25000
INFO:tensorflow:Finished evaluation at 2018-11-06-07:55:10
INFO:tensorflow:Saving dict for global step 25000: average_loss = 9520702000.0, global_step = 25000, loss = 95084180000.0


{'average_loss': 9520702000.0, 'loss': 95084180000.0, 'global_step': 25000}

# Great Job!